# Matching the functions

In [ ]:
from nbformat import reads, NO_CONVERT
from nbconvert import PythonExporter
def code_extractor(jpt):
    nb = reads(jpt, NO_CONVERT)
    exporter = PythonExporter()
    source, meta = exporter.from_notebook_node(nb)
    return source

In [ ]:
import imp
Matchmaker = imp.load_source('name', '../matchmaker.py')
import ast
def function_matching(nb):
    tree = ast.parse(nb)
    mm = Matchmaker.Matchmaker()
    mm.visit(tree)
    matchs = mm.matching()
    return matchs

In [ ]:
from pydriller import RepositoryMining, GitRepository
import difflib
import pandas

def repo_matching(repo, commit, path):
    gr = GitRepository(repo)
    for c in RepositoryMining(repo, single = commit).traverse_commits():
        for modified_files in commit.modifications:
            if modified_files.new_path == path:
                before = code_extractor(modified_files.source_code_before)
                lines_b = before.strip().splitlines()
                new = code_extractor(modified_files.source_code)
                lines_n = new.strip().splitlines()
                
                diff = difflib.unified_diff(lines_b, lines_n, fromfile='before', tofile='new', lineterm='', n=0)
                lines = list(diff)[2:]
                lineno = 0
                changes = pd.DataFrame(columns = ['repo','path','author','commit','line','code'])
                for line in lines:
                    prefix = '@@'
                    if line.startswith(prefix):
                        s = line[line.find("+"):]
                        try:
                            lineno = int(s[1:s.find(",")])
                        except:
                            lineno = int(s[1:s.find(" ")])
                    else:
                        if line.startswith("+"):
                            changes = changes.append({"repo": c.project_name, 
                                                      "path": modified_files.new_path,
                                                      "author": c.author.email,
                                                      "commit": c.hash,
                                                      "line": lineno,
                                                      "code": line}, ignore_index= True)
                            lineno += 1
                matches = function_matching(new)
                result = pd.merge(matches, changes, how = 'inner', on= "line")
                return result

In [ ]:
import pandas as pd
import ast
import csv
# incresing the csv field size
import ctypes
csv.field_size_limit(int(ctypes.c_ulong(-1).value // 2))
failed = []
fields = ['lineno','line','function','package','class','repo','file', 'author']
repo_path = "/home/ubuntu/repos/"
with open('../data/commits_bigquery.csv',"r", encoding="utf8") as csvfile:
    with open("../data/repo_function_matched.csv", "w", encoding="utf8") as newfile:
        data = csv.DictReader(csvfile)
        writer = csv.DictWriter(newfile, fieldnames=fields)
        writer.writeheader()
        for row in data:
            try:
                commit = row['commit']
                repo = row['repo']
                path = row['path']
                path = path.replace("/", "\\")
                repo = repo_path + repo
                matched = repo_matching(repo, commit, path)
                matched.to_csv(newfile, header=False, index= False)
            except KeyboardInterrupt:
                break
            except Exception as e: 
                failed.append(e)
                continue
import pickle

with open('../data/fails', 'wb') as fp:
    pickle.dump(failed, fp)

In [37]:
import os.path

if os.path.isfile('/home/ubuntu/test/test2/test3/test'):
    print ("File exist")
else:
    print ("File not exist")

File exist


In [8]:
import os
print(os.getcwd() + "\n")

/home/ubuntu/ipython_project_analyzer/notebooks



In [49]:
with open("home/ubuntu/clone_repos", "r") as sh:
    for row in sh:
        print(row)
        break

FileNotFoundError: [Errno 2] No such file or directory: 'home/ubuntu/clone_repos'

In [42]:
from pydriller import RepositoryMining, GitRepository
repo = "/home/ubuntu/repos/abulbasar/machine-learning"
for commit in RepositoryMining(repo).traverse_commits():
    print(commit.hash)

NoSuchPathError: /home/ubuntu/repos/abulbasar/machine-learning
